In [20]:
import csv

In [21]:
benchmark_base_path = r'C:\Users\dhutt\Desktop\SCHEDULING_MODEL\code\external_test_data\FJSSPinstances\\'
#path_30_min = r'C:\Users\huda\Documents\GitHub\scheduling_model\code\reworked_data_model\results\gurobi_results\formatted_results\30_min.txt'
path_30_min = r'C:\Users\dhutt\Desktop\SCHEDULING_MODEL\code\reworked_data_model\results\30min_gurobi.txt'


In [22]:
file = open(path_30_min, 'r')
file_content = file.readlines()
results_30_min = [x.split(';') for x in file_content]

In [23]:
def read_benchmark_file(source, instance):
    if source.startswith('0'):
        target_file = f'Behnke{instance}.fjs'
    elif source.startswith('1'):
        target_file = f'BrandimarteMk{instance}.fjs'
    elif source.startswith('2a'):
        target_file = f'HurinkSdata{instance}.fjs'
    elif source.startswith('2b'):
        target_file = f'HurinkEdata{instance}.fjs'
    elif source.startswith('2c'):
        target_file = f'HurinkRdata{instance}.fjs'
    elif source.startswith('2d'):
        target_file = f'HurinkVdata{instance}.fjs'
    elif source.startswith('3'):
        target_file = f'DPpaulli{instance}.fjs'
    elif source.startswith('4'):
        target_file = f'ChambersBarnes{instance}.fjs'
    elif source.startswith('5'):
        target_file = f'Kacem{instance}.fjs'
    elif source.startswith('6'):
        target_file = f'Fattahi{instance}.fjs'
    path = benchmark_base_path + f'{source}\\{target_file}'
    return open(path, 'r').readlines()

In [24]:
def translate(columns, limit):
    line = dict()
    line['source'] = columns[0]
    line['instance'] = int(columns[1])
    line['best_objective'] = float(columns[2])
    line['gap'] = float(columns[3])
    line['status'] = int(columns[4])
    line['n_explored'] = int(float(columns[5]))
    line['runtime'] = float(columns[6])
    line['overall_runtime'] = float(columns[7])
    line['setup_time'] = line['overall_runtime'] - line['runtime']
    # add information about the benchmark instance
    benchmark_file = read_benchmark_file(line['source'], line['instance'])
    benchmark_system = benchmark_file[0].split(' ')
    line['n_machines'] = int(benchmark_system[1])
    line['n_jobs'] = int(benchmark_system[0])
    line['average_workstations_per_operation'] = float(benchmark_system[2])
    jobs = benchmark_file[1:]
    line['n_operations'] = sum([int(x.split(' ')[0]) for x in jobs])
    line['average_operations_per_job'] = line['n_operations'] / line['n_jobs']
    line['beta_flexibility'] = line['average_workstations_per_operation']/line['n_machines']
    line['approximate_max_dissimilarity'] = line['n_operations'] + line['average_operations_per_job'] * line['average_workstations_per_operation']
    line['limit'] = limit
    return line    

In [25]:
data_30 = []

for columns in results_30_min[1:]:
    line = translate(columns, 30)
    data_30.append(line)

In [26]:
def get_entry(source, instance, data):
    for entry in data:
        if entry['source'] == source and entry['instance'] == instance:
            return entry

In [27]:
results = []
win = 0
draw = 0
loss = 0
with open(r'C:\Users\dhutt\Downloads\lit_best_known\lit_best_known.csv', newline='\n') as csvfile:
    reader = csv.reader(csvfile, delimiter=';')
    next(reader, None) #NOTE: skips header
    for row in reader:
        source = row[0]
        instance = int(row[1])
        lb = float(row[2])
        ub = float(row[3])
        optimal = True if int(row[4]) == 1 else False
        gurobi_result = get_entry(source, instance, data_30)
        gurobi_result['lit_lb'] = lb
        gurobi_result['lit_ub'] = ub
        gurobi_result['lit_optimal'] = optimal
        results.append([source, instance, lb, ub, optimal, gurobi_result['best_objective'], gurobi_result['status']])
        print(results[-1])
        if results[-1][3] > int(results[-1][5]+0.5):
            win+=1
        elif results[-1][3] == int(results[-1][5]+0.5):
            draw+=1
        else:
            loss+=1
print(f'Gurobi Win:{win}')
print(f'Gurobi Draw:{draw}')
print(f'Gurobi Loss:{loss}')

['0_BehnkeGeiger', 1, 70.0, 91.0, False, 90.0, 2]
['0_BehnkeGeiger', 2, 75.0, 91.0, False, 91.0, 2]
['0_BehnkeGeiger', 3, 79.0, 91.0, False, 91.0, 2]
['0_BehnkeGeiger', 4, 76.0, 97.0, False, 96.99999999019992, 9]
['0_BehnkeGeiger', 5, 71.0, 91.0, False, 90.99999999079995, 9]
['0_BehnkeGeiger', 6, 78.0, 131.0, False, 141.99999998569888, 9]
['0_BehnkeGeiger', 7, 84.0, 130.0, False, 144.99999998539982, 9]
['0_BehnkeGeiger', 8, 76.0, 128.0, False, 137.9999999860999, 9]
['0_BehnkeGeiger', 9, 74.0, 129.0, False, 129.99999995263633, 9]
['0_BehnkeGeiger', 10, 81.0, 133.0, False, 139.99999998589817, 9]
['0_BehnkeGeiger', 11, 163.0, 259.0, False, 293.9999999704994, 9]
['0_BehnkeGeiger', 12, 157.0, 251.0, False, 297.99999999999494, 9]
['0_BehnkeGeiger', 13, 160.0, 252.0, False, 277.999999944194, 9]
['0_BehnkeGeiger', 14, 164.0, 258.0, False, 318.99999993599977, 9]
['0_BehnkeGeiger', 15, 159.0, 262.0, False, 296.9997890316003, 9]
['0_BehnkeGeiger', 16, 327.0, 566.0, False, 600.9999999999559, 9]
['

In [28]:
import os
import inspect
from translation import FJSSPInstancesTranslator
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
read_path = r'C:\Users\dhutt\Desktop\SCHEDULING_MODEL\code\external_test_data\FJSSPinstances\\'


sources = ['0_BehnkeGeiger', '1_Brandimarte', '2a_Hurink_sdata', '2b_Hurink_edata', '2c_Hurink_rdata', '2d_Hurink_vdata', '3_DPpaulli', '4_ChambersBarnes', '5_Kacem', '6_Fattahi']

scores = []
source_scores = []
instances = []
for benchmark_source in sources:
    full_path = read_path + benchmark_source + '/'
    source_scores.append([])
    for i in range(len(os.listdir(full_path))):
        file_content : list[str] = read_benchmark_file(benchmark_source, i+1)
        values = file_content[0].split(' ')
        workstation_amount = int(values[1])
        average_workstations = float(values[2])
        n_operations = sum([int(x.split(' ')[0]) for x in file_content[1:]])
        #max_dissimilarity = get_approximate_max_dissimilarity(values[0], file_content[1:], values[2])

        #scores.append([f'{benchmark_source}{i+1}', average_workstations/workstation_amount, values[0], values[1], values[2], max_dissimilarity, n_operations, average_workstations])
        #source_scores[-1].append(scores[-1])
        instances.append((FJSSPInstancesTranslator().translate(benchmark_source, i+1), benchmark_source, i+1))
        instances[-1][0].id = f'{benchmark_source}{i+1}'        


In [29]:
operation_mins = dict()
for instance in instances:
    operation_mins[instance[0].id] = dict()
    for machine in instance[0].workstations.values():
        for operation in machine.tasks:
            if operation[0].id not in operation_mins[instance[0].id]:
                operation_mins[instance[0].id][operation[0].id] = float('inf')
            operation_mins[instance[0].id][operation[0].id] = min(operation_mins[instance[0].id][operation[0].id], machine.get_duration(operation[0]) if machine.get_duration(operation[0]) != 0 else float('inf'))
min_lengths = dict()
for instance in instances:
    if instance[0].id not in min_lengths:
        min_lengths[instance[0].id] = 0
    for recipe in instance[0].recipes.values():
        max_job = 0
        for operation in recipe.tasks:
            max_job += operation_mins[instance[0].id][operation[0].id]
    min_lengths[instance[0].id] = max(max_job, min_lengths[instance[0].id])

In [30]:
def get_list_element(source, instance, elements):
    for element in elements:
        if element[0] == source and element[1] == instance:
            return element

In [31]:
for instance in instances:
    entry = get_list_element(instance[1], instance[2], results)
    entry.append(min_lengths[instance[0].id])
    print(entry)

['0_BehnkeGeiger', 1, 70.0, 91.0, False, 90.0, 2, 59]
['0_BehnkeGeiger', 2, 75.0, 91.0, False, 91.0, 2, 61]
['0_BehnkeGeiger', 3, 79.0, 91.0, False, 91.0, 2, 76]
['0_BehnkeGeiger', 4, 76.0, 97.0, False, 96.99999999019992, 9, 69]
['0_BehnkeGeiger', 5, 71.0, 91.0, False, 90.99999999079995, 9, 54]
['0_BehnkeGeiger', 6, 78.0, 131.0, False, 141.99999998569888, 9, 61]
['0_BehnkeGeiger', 7, 84.0, 130.0, False, 144.99999998539982, 9, 62]
['0_BehnkeGeiger', 8, 76.0, 128.0, False, 137.9999999860999, 9, 69]
['0_BehnkeGeiger', 9, 74.0, 129.0, False, 129.99999995263633, 9, 67]
['0_BehnkeGeiger', 10, 81.0, 133.0, False, 139.99999998589817, 9, 60]
['0_BehnkeGeiger', 11, 163.0, 259.0, False, 293.9999999704994, 9, 65]
['0_BehnkeGeiger', 12, 157.0, 251.0, False, 297.99999999999494, 9, 77]
['0_BehnkeGeiger', 13, 160.0, 252.0, False, 277.999999944194, 9, 68]
['0_BehnkeGeiger', 14, 164.0, 258.0, False, 318.99999993599977, 9, 67]
['0_BehnkeGeiger', 15, 159.0, 262.0, False, 296.9997890316003, 9, 65]
['0_Behn

In [32]:
ga_path = r'C:\Users\huda\Documents\GitHub\scheduling_model\code\reworked_data_model\results\ga_results\small_gap_30min\\'
files = ['small_gap1.txt','small_gap2.txt','small_gap3.txt','small_gap4.txt','small_gap5.txt']

In [33]:
def translate(content, limit):
    line = dict()
    line['source'] = content[0]
    line['instance'] = int(content[1])
    line['best_objective'] = float(content[6])
    line['generations'] = int(content[4])
    line['restarts'] = int(content[5])
    #line['gap'] = float(columns[3])
    line['status'] = 2 if float(content[2]) < limit else 9 # NOTE: could still be optimal, just not known
    line['n_explored'] = int(float(content[3]))
    line['runtime'] = float(content[2])

    # add information about the benchmark instance
    benchmark_file = read_benchmark_file(line['source'], line['instance'])
    benchmark_system = benchmark_file[0].split(' ')
    line['n_machines'] = int(benchmark_system[1])
    line['n_jobs'] = int(benchmark_system[0])
    line['average_workstations_per_operation'] = float(benchmark_system[2])
    jobs = benchmark_file[1:]
    line['n_operations'] = sum([int(x.split(' ')[0]) for x in jobs])
    line['average_operations_per_job'] = line['n_operations'] / line['n_jobs']
    line['beta_flexibility'] = line['average_workstations_per_operation']/line['n_machines']
    line['approximate_max_dissimilarity'] = line['n_operations'] + line['average_operations_per_job'] * line['average_workstations_per_operation']
    line['limit'] = limit
    return line    

In [34]:
data = []
for file in files:
    content = open(ga_path + file, 'r').readlines()
    for values in content:
        line = translate(values.split(';'), 1800)
        list_entry = get_list_element(line['source'], line['instance'], results)
        list_entry.append(line['best_objective'])
        data.append(line)

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\huda\\Documents\\GitHub\\scheduling_model\\code\\reworked_data_model\\results\\ga_results\\small_gap_30min\\\\small_gap1.txt'

In [ ]:
print(results)

[['0_BehnkeGeiger', 1, 70.0, 91.0, False, 90.0, 2, 98.0, 100.0, 104.0, 100.0, 99.0], ['0_BehnkeGeiger', 2, 75.0, 91.0, False, 91.0, 2, 104.0, 105.0, 101.0, 103.0, 103.0], ['0_BehnkeGeiger', 3, 79.0, 91.0, False, 91.0, 2, 106.0, 106.0, 104.0, 105.0, 105.0], ['0_BehnkeGeiger', 4, 76.0, 97.0, False, 96.99999999019992, 9, 110.0, 105.0, 107.0, 110.0, 106.0], ['0_BehnkeGeiger', 5, 71.0, 91.0, False, 90.99999999079995, 9, 102.0, 102.0, 103.0, 102.0, 98.0], ['0_BehnkeGeiger', 6, 78.0, 131.0, False, 141.99999998569888, 9], ['0_BehnkeGeiger', 7, 84.0, 130.0, False, 144.99999998539982, 9], ['0_BehnkeGeiger', 8, 76.0, 128.0, False, 137.9999999860999, 9], ['0_BehnkeGeiger', 9, 74.0, 129.0, False, 129.99999995263633, 9], ['0_BehnkeGeiger', 10, 81.0, 133.0, False, 139.99999998589817, 9], ['0_BehnkeGeiger', 11, 163.0, 259.0, False, 293.9999999704994, 9], ['0_BehnkeGeiger', 12, 157.0, 251.0, False, 297.99999999999494, 9], ['0_BehnkeGeiger', 13, 160.0, 252.0, False, 277.999999944194, 9], ['0_BehnkeGeige

In [ ]:
ga_win = 0
ga_gurobi_win = 0
ga_gurobi_draw = 0
ga_gurobi_loss = 0
ga_draw = 0
ga_loss = 0
#NOTE: only 189 GA results, wait for rest TODO: rerun 189 with correct settings
for result in results:
    if result[3] < result[6]:
        ga_win += 1
    elif results[3] == results[6]:
        ga_draw += 1
    else:
        ga_loss += 1
    if int(result[5]+0.5) < result[6]:
        ga_gurobi_win+=1
    elif int(result[5]+0.5) == result[6]:
        ga_gurobi_draw+=1
    else:
        ga_gurobi_loss+=1
print(ga_win)
print(ga_draw)
print(ga_loss)
print(ga_gurobi_win)
print(ga_gurobi_draw)
print(ga_gurobi_loss)

66
0
336
0
0
402
